<a href="https://colab.research.google.com/github/ouahenre/Jupiter_notebook_exercices_for_python_pandas_numpy/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Analyse de données sur les crimes ayant été commis à Chicago depuis 2001 à nos jours août 2024. Le fichier CSV téléchargé comporte plus de 8 millions de lignes.
On utilisera Pyspark pour sa rapidité sur Pandas

In [ ]:
pip install pyspark -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
pip install findspark -q


In [ ]:
#création d'une session Spark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Pyspark project').getOrCreate()

sample_data/


source de données https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-Present/ijzp-q8t2/about_data

In [ ]:
Téléchargement du fichier csv


In [ ]:
!wget https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD

--2024-08-22 21:20:32--  https://data.cityofchicago.org/api/views/ijzp-q8t2/rows.csv?accessType=DOWNLOAD
Resolving data.cityofchicago.org (data.cityofchicago.org)... 52.206.140.199, 52.206.140.205, 52.206.68.26
Connecting to data.cityofchicago.org (data.cityofchicago.org)|52.206.140.199|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘rows.csv?accessType=DOWNLOAD’

rows.csv?accessType     [              <=>   ]   1.79G  2.86MB/s    in 11m 22s 

2024-08-22 21:31:54 (2.69 MB/s) - ‘rows.csv?accessType=DOWNLOAD’ saved [1921893857]



In [ ]:
ls

 reported-crimes.csv  'rows.csv?accessType=DOWNLOAD'   sample_data/


renommage du fichier

In [ ]:
!mv rows.csv?accessType=DOWNLOAD reported-crimes.csv

In [ ]:
ls

reported-crimes.csv  sample_data/


Ingestion de données dans le dataframe

In [ ]:
#lecture du fichier
df=spark.read.csv('reported-crimes.csv',
                  header=True)
df.show(5)

+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|                Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+--------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11037294|   JA371270|03/18/2015 12:00:...|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11| 

In [ ]:
df.count()

8136299

In [ ]:
#Panda prend 1 mn pour l'import en dataframe et l'affichage des 5 1res lignes
#Or Pyspark prend 1 s
import pandas as pd
df_pandas=pd.read_csv('reported-crimes.csv')
df_pandas.head(5)

,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,11037294,JA371270,03/18/2015 12:00:00 PM,0000X W WACKER DR,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,BANK,False,False,...,42.0,32.0,11,NaN,NaN,2015,08/01/2017 03:52:26 PM,NaN,NaN,NaN
1,11646293,JC213749,12/20/2018 03:00:00 PM,023XX N LOCKWOOD AVE,1154,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT $300 AND UNDER,APARTMENT,False,False,...,36.0,19.0,11,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
2,11645836,JC212333,05/01/2016 12:25:00 AM,055XX S ROCKWELL ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,False,False,...,15.0,63.0,11,NaN,NaN,2016,04/06/2019 04:04:43 PM,NaN,NaN,NaN
3,11645959,JC211511,12/20/2018 04:00:00 PM,045XX N ALBANY AVE,2820,OTHER OFFENSE,TELEPHONE THREAT,RESIDENCE,False,False,...,33.0,14.0,08A,NaN,NaN,2018,04/06/2019 04:04:43 PM,NaN,NaN,NaN
4,11645601,JC212935,06/01/2014 12:01:00 AM,087XX S SANGAMON ST,1153,DECEPTIVE PRACTICE,FINANCIAL IDENTITY THEFT OVER $ 300,RESIDENCE,False,False,...,21.0,71.0,11,NaN,NaN,2014,04/06/2019 04:04:43 PM,NaN,NaN,NaN


In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
#type des variables
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



Manipulation des données

In [ ]:
from pyspark.sql.functions import to_timestamp,col
df=df.withColumn(
    'Date',
    to_timestamp(col('Date'),'MM/dd/yyyy hh:mm:ss a'))
df.printSchema()

root
 |-- ID: string (nullable = true)
 |-- Case Number: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- Block: string (nullable = true)
 |-- IUCR: string (nullable = true)
 |-- Primary Type: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Location Description: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: string (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- Community Area: string (nullable = true)
 |-- FBI Code: string (nullable = true)
 |-- X Coordinate: string (nullable = true)
 |-- Y Coordinate: string (nullable = true)
 |-- Year: string (nullable = true)
 |-- Updated On: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Location: string (nullable = true)



In [ ]:
df.show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|      11|     

In [ ]:
#affichage de colonnes particulières
df.select('Description','Date','Arrest').show(5)

+--------------------+-------------------+------+
|         Description|               Date|Arrest|
+--------------------+-------------------+------+
|FINANCIAL IDENTIT...|2015-03-18 12:00:00| false|
|FINANCIAL IDENTIT...|2018-12-20 15:00:00| false|
|FINANCIAL IDENTIT...|2016-05-01 00:25:00| false|
|    TELEPHONE THREAT|2018-12-20 16:00:00| false|
|FINANCIAL IDENTIT...|2014-06-01 00:01:00| false|
+--------------------+-------------------+------+
only showing top 5 rows



In [ ]:
#Créer une nouvelle colonne et l'initialiser à 1
from pyspark.sql.functions import lit
df=df.withColumn('One',lit(1))
df.show(5)

+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|      ID|Case Number|               Date|               Block|IUCR|      Primary Type|         Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|Latitude|Longitude|Location|One|
+--------+-----------+-------------------+--------------------+----+------------------+--------------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+--------+---------+--------+---+
|11037294|   JA371270|2015-03-18 12:00:00|   0000X W WACKER DR|1153|DECEPTIVE PRACTICE|FINANCIAL IDENTIT...|                BANK| false|   false|0111|     001|  42|            32|  

In [ ]:
#faire des filtration sur des lignes
#crimes signalés le 25-12-2023
one_day=df.filter(df.Date=='2023-12-25')
one_day.count()


24

In [ ]:
one_day2=df.filter(col('Date')=='2023-12-25')
one_day2.count()

24

In [ ]:
one_day==one_day2

False

In [ ]:
one_day.distinct().count()

24

In [50]:
#grouper selon la colonne Primary Type
df.groupBy('Primary Type').count().show()

+--------------------+-------+
|        Primary Type|  count|
+--------------------+-------+
|OFFENSE INVOLVING...|  58345|
|CRIMINAL SEXUAL A...|   9201|
|            STALKING|   5473|
|PUBLIC PEACE VIOL...|  53558|
|           OBSCENITY|    881|
|               ARSON|  13926|
|            GAMBLING|  14643|
|   CRIMINAL TRESPASS| 220561|
|             ASSAULT| 538329|
|LIQUOR LAW VIOLATION|  15147|
| MOTOR VEHICLE THEFT| 409630|
|               THEFT|1720951|
|             BATTERY|1483803|
|             ROBBERY| 306443|
|            HOMICIDE|  13388|
|           RITUALISM|     24|
|    PUBLIC INDECENCY|    203|
| CRIM SEXUAL ASSAULT|  27455|
|   HUMAN TRAFFICKING|    109|
|        INTIMIDATION|   4927|
+--------------------+-------+
only showing top 20 rows



In [51]:
#Quel est le pourcentage de crimes signalées depuis 2001 qui ont conduit à des arrestations
df.filter(df.Arrest=='true').count()/df.select('Arrest').count()

0.2556468241887374

In [53]:
#QUels sont les 3 principaux lieux pour les crimes signalés
n_location=df.groupBy('Location Description').count()
n_location.show()

+--------------------+------+
|Location Description| count|
+--------------------+------+
|SCHOOL - PRIVATE ...|  1145|
|AIRPORT TERMINAL ...|  2545|
|VEHICLE - COMMERCIAL|   973|
|POLICE FACILITY/V...| 18566|
|RESIDENCE - YARD ...| 12673|
|CHA PARKING LOT /...|  1966|
|            SIDEWALK|747443|
|AIRPORT TERMINAL ...|   134|
|OTHER RAILROAD PR...|   535|
|CTA GARAGE / OTHE...| 10276|
|            CAR WASH|  3496|
|    AIRPORT/AIRCRAFT| 16296|
|            HOSPITAL|    16|
|MEDICAL/DENTAL OF...|  7430|
|    FEDERAL BUILDING|  1023|
|         CTA STATION|  7382|
|SCHOOL, PUBLIC, G...| 30252|
|SPORTS ARENA/STADIUM|  5290|
|                FARM|     9|
|            CEMETARY|   431|
+--------------------+------+
only showing top 20 rows



In [55]:
n_location.orderBy('count',ascending=False).show(3)

+--------------------+-------+
|Location Description|  count|
+--------------------+-------+
|              STREET|2126465|
|           RESIDENCE|1348146|
|           APARTMENT| 944976|
+--------------------+-------+
only showing top 3 rows



In [63]:
#crime moins signalé
df.groupBy('Primary Type').count().orderBy('count',ascending=True).show(10,truncate=False)

+---------------------------------+-----+
|Primary Type                     |count|
+---------------------------------+-----+
|DOMESTIC VIOLENCE                |1    |
|NON-CRIMINAL (SUBJECT SPECIFIED) |9    |
|RITUALISM                        |24   |
|NON - CRIMINAL                   |38   |
|HUMAN TRAFFICKING                |109  |
|OTHER NARCOTIC VIOLATION         |153  |
|NON-CRIMINAL                     |185  |
|PUBLIC INDECENCY                 |203  |
|OBSCENITY                        |881  |
|CONCEALED CARRY LICENSE VIOLATION|1352 |
+---------------------------------+-----+
only showing top 10 rows



In [64]:
#activité non criminelle la moins signalée
df.filter(df['Primary Type']=='NON - CRIMINAL').groupBy('Description').count().orderBy('count',ascending=False).show(truncate=False)

+-----------------+-----+
|Description      |count|
+-----------------+-----+
|FOID - REVOCATION|38   |
+-----------------+-----+



In [65]:
#activité criminelle la moins signalée
df.filter(df['Primary Type']=='CRIMINAL').groupBy('Description').count().orderBy('count',ascending=False).show(truncate=False)

+-----------+-----+
|Description|count|
+-----------+-----+
+-----------+-----+



In [70]:
from os import truncate
#activité criminelle la plus signaléé
df.groupBy('Primary Type').count().orderBy('count',ascending=False).show(truncate=False)

+--------------------------------+-------+
|Primary Type                    |count  |
+--------------------------------+-------+
|THEFT                           |1720951|
|BATTERY                         |1483803|
|CRIMINAL DAMAGE                 |926615 |
|NARCOTICS                       |755364 |
|ASSAULT                         |538329 |
|OTHER OFFENSE                   |505410 |
|BURGLARY                        |434461 |
|MOTOR VEHICLE THEFT             |409630 |
|DECEPTIVE PRACTICE              |368568 |
|ROBBERY                         |306443 |
|CRIMINAL TRESPASS               |220561 |
|WEAPONS VIOLATION               |117798 |
|PROSTITUTION                    |70144  |
|OFFENSE INVOLVING CHILDREN      |58345  |
|PUBLIC PEACE VIOLATION          |53558  |
|SEX OFFENSE                     |32614  |
|CRIM SEXUAL ASSAULT             |27455  |
|INTERFERENCE WITH PUBLIC OFFICER|19211  |
|LIQUOR LAW VIOLATION            |15147  |
|GAMBLING                        |14643  |
+----------

In [71]:
#Dataframe des activités non-criminal
df_non_criminal=df.filter(df['Primary Type']=='NON - CRIMINAL')
df_non_criminal.show(5)

+-------+-----------+-------------------+--------------------+----+--------------+-----------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|     ID|Case Number|               Date|               Block|IUCR|  Primary Type|      Description|Location Description|Arrest|Domestic|Beat|District|Ward|Community Area|FBI Code|X Coordinate|Y Coordinate|Year|          Updated On|    Latitude|    Longitude|            Location|One|
+-------+-----------+-------------------+--------------------+----+--------------+-----------------+--------------------+------+--------+----+--------+----+--------------+--------+------------+------------+----+--------------------+------------+-------------+--------------------+---+
|9284607|   HW428723|2013-08-29 09:30:00|    007XX E 111TH ST|5114|NON - CRIMINAL|FOID - REVOCATION|POLICE FACILITY/V...| false|   false|0531|   

In [72]:
#fonctions de pyspark
from pyspark.sql import functions as F
dir(F)

['Any',
 'ArrayType',
 'Callable',
 'Column',
 'DataFrame',
 'DataType',
 'Dict',
 'Iterable',
 'JVMView',
 'List',
 'Optional',
 'PandasUDFType',
 'PySparkTypeError',
 'PySparkValueError',
 'SparkContext',
 'StringType',
 'StructType',
 'TYPE_CHECKING',
 'Tuple',
 'Type',
 'Union',
 'UserDefinedFunction',
 'UserDefinedTableFunction',
 'ValuesView',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_create_column_from_literal',
 '_create_lambda',
 '_create_py_udf',
 '_create_py_udtf',
 '_from_numpy_type',
 '_get_jvm_function',
 '_get_lambda_parameters',
 '_invoke_binary_math_function',
 '_invoke_function',
 '_invoke_function_over_columns',
 '_invoke_function_over_seq_of_columns',
 '_invoke_higher_order_function',
 '_options_to_str',
 '_test',
 '_to_java_column',
 '_to_seq',
 '_unresolved_named_lambda_variable',
 'abs',
 'acos',
 'acosh',
 'add_months',
 'aes_decrypt',
 'aes_encrypt',
 'aggregate',
 'any_value',
 'approxC

In [73]:
len(dir(F))

484